In [1]:
import sys,os
sys.path.insert(0, os.path.join(os.path.abspath(''), "YOLOX"))

import numpy as np
import cv2
import imutils

from tracker import Tracker
from pose_detector import PoseDetector
from sequence_detector import SequenceDetector
from pose_tracker import PoseTracker
from visualizer import Visualizer
from project_utils import identify_bbox

environment = "local"

In [2]:
if environment == "colab":
    from IPython.display import display, Javascript, Image, clear_output
    from base64 import b64decode, b64encode
    import cv2, PIL, io
     # Import PyDrive and associated libraries.
    # This only needs to be done once per notebook.
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials

    # Authenticate and create the PyDrive client.
    # This only needs to be done once per notebook.
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)

    !gdown --id 1iYxlkO_RZJ_6iL6wdIbfjl8VprLlFp1c

         # function to convert the JavaScript object into an OpenCV image
    def js_to_image(js_reply):
        """
        Params:
                js_reply: JavaScript object containing image from webcam
        Returns:
                img: OpenCV BGR image
        """
        # decode base64 image
        image_bytes = b64decode(js_reply.split(',')[1])
        # convert bytes to numpy array
        jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
        # decode numpy array into OpenCV BGR image
        img = cv2.imdecode(jpg_as_np, flags=1)

        return img

    # function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
    def bbox_to_bytes(bbox_array):
        """
        Params:
                bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
        Returns:
              bytes: Base64 image byte string
        """
        # convert array into PIL image
        bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
        iobuf = io.BytesIO()
        # format bbox into png for return
        bbox_PIL.save(iobuf, format='png')
        # format return string
        bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

        return bbox_bytes
else:
    cap = cv2.VideoCapture(0)

In [3]:
from pose_tracker import PoseTracker
tracker = Tracker('person')
pose_detector = PoseDetector(n_feature=8, n_output=3, checkpoint_path="pose_classification_simplified.ckpt", simple_mode=True, model_based=False)
sequence_detector = SequenceDetector([1,2], 10)
pose_tracker = PoseTracker(pose_detector, sequence_detector, tracker)
visualizer = Visualizer(environment)
poi = None


In [4]:
if environment == "colab":
    # start streaming video from webcam
    video_stream()
    # label for video
    label_html = 'Capturing...'
    # initialze bounding box to empty
    bbox = ''
    #count = 0
while True:
    if environment == "colab":
        js_reply = video_frame(label_html, bbox)
        if not js_reply:
            break
        # convert JS response to OpenCV Image
        img = js_to_image(js_reply["img"])
        # create transparent overlay for bounding box
        bbox_array = np.zeros([480,640,4], dtype=np.uint8)
    else:
        _, img = cap.read()
        if img is None:
            break

    poi = None
    im = imutils.resize(img, height=500)
    #bbox, pose = pose_detector.inference(im)
    outputs, poi = pose_tracker.update(im)
    visualizer.show(im, outputs, poi)

C:\Users\agath\anaconda3\lib\site-packages\torch\functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


NotImplementedError: Could not run 'torchvision::nms' with arguments from the 'CUDA' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'torchvision::nms' is only available for these backends: [CPU, QuantizedCPU, BackendSelect, Python, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradXLA, AutogradLazy, AutogradXPU, AutogradMLC, AutogradHPU, Tracer, AutocastCPU, Autocast, Batched, VmapMode, Functionalize].

CPU: registered at C:\Users\circleci\project\torchvision\csrc\ops\cpu\nms_kernel.cpp:112 [kernel]
QuantizedCPU: registered at C:\Users\circleci\project\torchvision\csrc\ops\quantized\cpu\qnms_kernel.cpp:125 [kernel]
BackendSelect: fallthrough registered at C:\cb\pytorch_1000000000000\work\aten\src\ATen

In [ ]:
outputs.shape